In [1]:
!pip install -q transformers[torch] datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 27.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
# mount your google drive
drive.mount('/content/drive')

# cambiamos el directorio de trabajo a la carpeta donde está almacenado el dataset
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DIPROMATS_2024/data/')

Mounted at /content/drive


In [3]:
import pandas as pd

#df_train_en = pd.read_json("dipromats24_t1_train_en.json")
df_train_es = pd.read_json("dipromats24_t1_train_es.json")

#print('dataset size:', df_train_en.shape[0] + df_train_es.shape[0])
print('dataset size:', df_train_es.shape)
#print('english dataset size:', df_train_en.shape[0])
print('spanish dataset size:', df_train_es.shape)

#show the first rows
#df_train_en.head()
df_train_es.head()

dataset size: (6120, 13)
spanish dataset size: (6120, 13)


,test_case,id,country,username,tweet_type,tweet_id,UTC,rts&fav,language,text,label_task1,label_task2,label_task3
0,DIPROMATS2024-TASK1,0,Russia,embrusiamexico,Reply,1212162072710131712,2020-01-01 00:02:13+00:00,6,es,@mae_rusia @mbellasartes @PalacioOficial @mfa_...,False,[],[]
1,DIPROMATS2024-TASK1,1,Russia,embrusiamexico,Reply,1212164441191649280,2020-01-01 00:11:38+00:00,5,es,@mae_rusia @mbellasartes @PalacioOficial @mfa_...,False,[],[]
2,DIPROMATS2024-TASK1,2,USA,usembassymex,Tweet,1212252277068746752,2020-01-01 06:00:40+00:00,377,es,"Felicidad, paz, prosperidad y todos nuestros m...",True,[1 appeal to commonality],[1 appeal to commonality - flag waving]
3,DIPROMATS2024-TASK1,3,Russia,embrusiaenarges,Tweet,1212400271466389504,2020-01-01 15:48:44+00:00,43,es,Queridos amigos!\n\nDesde la Embajada de #Rusi...,False,[],[]
4,DIPROMATS2024-TASK1,4,USA,usembassyve,Tweet,1212481293100945408,2020-01-01 21:10:41+00:00,1082,es,Desde todo el equipo de la Oficina Externa de ...,False,[],[]


In [4]:
#df_train_en = df_train_en[['country', 'tweet_type', 'text', 'label_task1']] # we ignore the other columns
df_train_es = df_train_es[['country', 'tweet_type', 'text', 'label_task1']] # we ignore the other columns

df_train_es.head()


,country,tweet_type,text,label_task1
0,Russia,Reply,@mae_rusia @mbellasartes @PalacioOficial @mfa_...,False
1,Russia,Reply,@mae_rusia @mbellasartes @PalacioOficial @mfa_...,False
2,USA,Tweet,"Felicidad, paz, prosperidad y todos nuestros m...",True
3,Russia,Tweet,Queridos amigos!\n\nDesde la Embajada de #Rusi...,False
4,USA,Tweet,Desde todo el equipo de la Oficina Externa de ...,False


In [5]:
from sklearn.model_selection import train_test_split

X=df_train_es.drop(['label_task1'], axis=1)
y=df_train_es['label_task1']

X_train, X_aux, y_train, y_aux = train_test_split(X, y,
        test_size=0.3, random_state=42, stratify=y)

print('Training set size:', len(X_train))
print('Rest :', len(X_aux))

Training set size: 4284
Rest : 1836


In [6]:
X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux,
        test_size=1/2, random_state=42, stratify=y_aux)

print('Tamaño conjunto de entrenamiento:', len(X_train))
print('Tamaño conjunto de validación:', len(X_val))
print('Tamaño conjunto de evaluación:', len(X_test))

Tamaño conjunto de entrenamiento: 4284
Tamaño conjunto de validación: 918
Tamaño conjunto de evaluación: 918


In [7]:
df_train_gen = pd.DataFrame()
df_train_gen = df_train_gen.assign(text=X_train["text"].values)
df_train_gen = df_train_gen.assign(label=y_train.values)

print(df_train_gen)

                                                   text  label
0     En la cuestión del origen del virus, la distan...   True
1     Hoy hemos debatido sobre el papel de la UE en ...  False
2     Conversación telefónica entre los mandatarios ...  False
3     #Economía| #China🇨🇳 como uno de los principale...  False
4     @weiasecas Mi solidaridad con el pueblo de Chi...  False
...                                                 ...    ...
4279  #TeamEurope 🇪🇺🇲🇽 | Para enfrentar los retos de...  False
4280  Primer Ministro Li Keqiang dijo: China es un p...  False
4281  La VAU ha recibido información creíble que los...  False
4282  #EsNoticia || Jefe de Estado, @NicolasMaduro: ...  False
4283  Nuestra más sincera gratitud a la ministra @Ar...  False

[4284 rows x 2 columns]


In [8]:
df_val_gen = pd.DataFrame()
df_val_gen = df_val_gen.assign(text=X_val["text"].values)
df_val_gen = df_val_gen.assign(label=y_val.values)

print(df_val_gen)

                                                  text  label
0    Un gran privilegio unirme a @SecPompeo en su v...  False
1    .@Nelfuent: "La relación de El Salvador y la #...  False
2    (1/4) "La rapidez del gobierno chino en la con...  False
3    Como líder de la Asamblea Nacional, Guaidó sig...  False
4    El gobierno y el pueblo chino están concentrad...  False
..                                                 ...    ...
913  El próximo lunes se inaugura en el @whitneymus...  False
914  .@SecPompeo: Condenamos enérgicamente abusos d...   True
915  El Embajador de Rusia en Guatemala, Alexander ...  False
916  Los buques petroleros iraníes que llegan a #Ve...  False
917  🔊🇪🇺🏳️‍🌈Declaración HRVP @JosepBorrellF con mot...  False

[918 rows x 2 columns]


In [9]:
from transformers import (
    XLNetConfig,
    XLNetTokenizer,
    XLNetForSequenceClassification,
    TrainingArguments,
    pipeline,
    set_seed
)

# Establecer semilla para reproducibilidad.
set_seed(123)

# Número de épocas de entrenamiento.
epochs = 4

# Tamaño del lote - dependiendo de la longitud máxima de la secuencia y la memoria GPU.
batch_size = 32

# Longitud máxima de las secuencias de texto.
MAX_LENGTH = 50

# Nombre o ruta del modelo pre-entrenado.
model_name_or_path = "xlnet/xlnet-base-cased"

# Diccionario de etiquetas y sus identificadores.
labels_ids = {'False': 0, 'True': 1}

# Número de etiquetas que se están utilizando en el entrenamiento.
n_labels = len(labels_ids)

# Right Padding

In [10]:
model_config = XLNetConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

tokenizer = XLNetTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [11]:
model = XLNetForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
df_train_gen[:10]

,text,label
0,"En la cuestión del origen del virus, la distan...",True
1,Hoy hemos debatido sobre el papel de la UE en ...,False
2,Conversación telefónica entre los mandatarios ...,False
3,#Economía| #China🇨🇳 como uno de los principale...,False
4,@weiasecas Mi solidaridad con el pueblo de Chi...,False
5,#SOTEU establecerá la guía hacia el futuro de ...,False
6,#EsNoticia || Trump ataca a Venezuela en su de...,True
7,‼️⚠️ATENCIÓN PERIODISTAS ⚠️‼️👇\nEn este hilo o...,True
8,#EsNoticia || En ocasión del Día Nacional del ...,False
9,Embajadora Stella Zervoudaki: “La Unión Europe...,False


In [13]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train_gen)
val_dataset = Dataset.from_pandas(df_val_gen)

In [14]:
tokenized_ids = [labels_ids[str(label)] for label in train_dataset["label"]]
print(tokenized_ids)
train_dataset = train_dataset.remove_columns("label").add_column("label", tokenized_ids)

[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [15]:
tokenized_ids = [labels_ids[str(label)] for label in val_dataset["label"]]
print(tokenized_ids)
val_dataset = val_dataset.remove_columns("label").add_column("label", tokenized_ids)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [16]:
def tokenize(examples):
    return tokenizer(examples["text"], return_tensors="pt", padding="max_length", truncation=True, max_length=MAX_LENGTH)

In [17]:
encoded_data_train = train_dataset.map(tokenize, batched=True)
encoded_data_train

Map:   0%|          | 0/4284 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4284
})

In [18]:
encoded_data_val = val_dataset.map(tokenize, batched=True)
encoded_data_val

Map:   0%|          | 0/918 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 918
})

In [19]:
from transformers import TrainingArguments
args = TrainingArguments(output_dir="./outputs")
args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_

In [20]:
args.per_device_train_batch_size = 16
args.per_device_eval_batch_size = 16

In [21]:
args.evaluation_strategy="epoch"

In [22]:
from huggingface_hub import HfFolder, notebook_login

notebook_login()

In [23]:
repository_id = "MASE98/finetune_right_padding_XLNet_base_cased"

training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

In [24]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):

    y_true = pred.label_ids                 # son las labels reales
    y_pred = pred.predictions.argmax(-1)    # son las predicciones


    acc = accuracy_score(y_true, y_pred)

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [25]:
from transformers import Trainer

trainer = Trainer(
    model = model,# modelo que será ajustado
    train_dataset = encoded_data_train,# conjunto training
    eval_dataset = encoded_data_val,# conjunto de validación

    args = training_args,# hiperparámetros
    compute_metrics=compute_metrics,# función para computar las métricas
)

In [26]:
import torch
torch.cuda.empty_cache()

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.523600,0.567581,0.803922,0.445652,0.401961,0.500000
2,0.512400,0.494474,0.803922,0.445652,0.401961,0.500000
3,0.319800,0.490530,0.803922,0.445652,0.401961,0.500000
4,0.429000,0.444074,0.816993,0.643926,0.704393,0.623645


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2144, training_loss=0.4834792228109801, metrics={'train_runtime': 355.4955, 'train_samples_per_second': 48.203, 'train_steps_per_second': 6.031, 'total_flos': 476729342812800.0, 'train_loss': 0.4834792228109801, 'epoch': 4.0})

In [28]:
trainer.evaluate()

{'eval_loss': 0.44407370686531067,
 'eval_accuracy': 0.8169934640522876,
 'eval_f1': 0.6439264506238398,
 'eval_precision': 0.7043927327028372,
 'eval_recall': 0.6236449864498645,
 'eval_runtime': 4.3877,
 'eval_samples_per_second': 209.222,
 'eval_steps_per_second': 26.21,
 'epoch': 4.0}